In [1]:
from elasticsearch import Elasticsearch, helpers
from datetime import datetime
import pandas as pd

In [2]:
es = Elasticsearch("http://localhost:9400",basic_auth=("elastic","secret"),verify_certs=False)
es.indices.get_alias(index="*")

ObjectApiResponse({'education': {'aliases': {}}, 'data_description': {'aliases': {}}, 'achievements': {'aliases': {}}, 'activity': {'aliases': {}}, 'log': {'aliases': {}}, 'appsettings': {'aliases': {}}, 'exercise': {'aliases': {}}, 'baseline': {'aliases': {}}, 'plan': {'aliases': {}}, 'account': {'aliases': {}}})

In [14]:
#engang
def complete_goal(userid,goalid):
    #possible goal ids
    #SetGoalSetting,SetSleepTool,UseMindfulness,QACompleted,SessionCompleted
    res = es.get(index="achievements", 
          id=userid+"_"+goalid)
    print(res)
    id=res["_id"]
    res=res["_source"]
    res["progress"]=1
    res["achievedat"]=1
    es.update(index='achievements',id=id,
                doc=res)


In [15]:
complete_goal("thea","SetGoalSetting")

{'_index': 'achievements', '_id': 'thea_SetGoalSetting', '_version': 1, '_seq_no': 0, '_primary_term': 1, 'found': True, '_source': {'userid': 'thea', 'achievementid': 'SetGoalSetting', 'progress': 0, 'goal': 1, 'achievedat': -1}}


In [21]:
def complete_quiz(userid):
    correct=es.search(index="education",query={"bool":
                                          {"must":[
                                              {"match":{"is_correct":True}},
                                                {"match":{"userid":"carmela"}}
                                          ]
                                         
                                          }}
    ,size=900).body["hits"]["hits"]
    num_correct=len(correct)
    for i in [1,2,7,14,25]:
        if num_correct>=i:
          complete_goal(userid,"EducationalCorrectQuiz"+str(i))
        else:
           break

In [35]:
def total_steps(userid):
    res = es.search(index="activity", query={'match' : {"userid":userid}},size=10000)["hits"]["hits"]
    steps=list(map(lambda x: x["_source"]["steps"],res))
    total_steps=sum(steps)
    for i in [42.195,50,100,200,400,600,800,1000]:
        if total_steps>=i*1000:
            if i == 42.195:
                complete_goal(userid,"Marathon")
            else:
                complete_goal(userid,"TotalSteps"+str(i))
        else:
           break

In [34]:
total_steps("thea")

In [8]:
#gjennomfør alle 3 aktivitetene(antall ganger)
#1,3,7,14,25,40

In [9]:
#korrekt svar på quiz(antall ganger)
#1,3,7,14,25

In [10]:
#lest godt å vite

In [11]:
datetime.fromtimestamp(1721483520000/1000).isoformat()

'2024-07-20T15:52:00'

In [12]:
res_achievements = es.search(index="achievements", query={'match' : {"userid":"user"}},size=10000)["hits"]["hits"][0]

In [13]:
completed_achievements=[]
res = es.search(index="activity", query={'match' : {"userid":"user"}},size=10000)["hits"]["hits"]
res=list(map(lambda x: x["_source"],res))
df=pd.DataFrame(res)
df.start=df.start.apply(lambda x: datetime.fromtimestamp(x/1000))
df.end=df.end.apply(lambda x: datetime.fromtimestamp(x/1000))
df

,start,end,type,steps,userid
0,2024-07-21 13:40:00,2024-07-21 14:28:00,current,14000,user
1,2024-07-15 10:00:00,2024-07-15 11:00:00,walking,200,user
2,2024-07-16 11:00:00,2024-07-16 12:00:00,walking,196,user
3,2024-07-18 10:00:00,2024-07-18 11:00:00,walking,2000,user
4,2024-07-18 03:00:00,2024-07-18 04:00:00,walking,400,user
5,2024-07-19 11:00:00,2024-07-19 12:00:00,walking,5679,user
6,2024-07-20 15:00:00,2024-07-20 16:00:00,walking,2000,user
7,2024-07-21 14:00:00,2024-07-21 15:00:00,walking,6000,user
8,2024-07-21 13:00:00,2024-07-21 14:00:00,walking,8000,user
9,2024-08-12 15:01:00,2024-08-12 15:02:00,current,102000,user


In [95]:
#max steps per day
def max_steps_per_day(userid,df):
    steps_per_day=df.groupby(df.start.dt.floor('1D')).steps.sum()
    for i in [10,12,14,16,18,20]:
        res = es.search(index="achievements", 
            query={"bool":{"must":[
                {'match' : 
                    {"userid":userid}},
                {'match' : 
                    {"achievementid":"DailyRecordSteps"+str(i)}}
            ]}},size=10000)["hits"]["hits"][0]
        id=res["_id"]
        res=res["_source"]
        res["progress"]=steps_per_day.max()
        if steps_per_day.max()>=i*1000:
            res["achievedat"]=1
        es.update(index='achievements',id=id,doc=res)

In [18]:
def total_steps(userid):
    completed_achievements=[]
    res = es.search(index="activity", query={'match' : {"userid":userid}},size=10000)["hits"]["hits"]
    res=list(map(lambda x: x["_source"],res))
    df=pd.DataFrame(res)
    df.start=df.start.apply(lambda x: datetime.fromtimestamp(x/1000))
    df.end=df.end.apply(lambda x: datetime.fromtimestamp(x/1000))
    df
    display(df)
    for i in [50,100,200,400,600,800,1000,42.195]:
        if i!=42.195:
            ach_id="TotalSteps"+str(i)
        else:
            ach_id="Marathon"
        res = es.search(index="achievements", 
            query={"bool":{"must":[
                {'match' : 
                    {"userid":userid}},
                {'match' : 
                    {"achievementid":ach_id}}
            ]}},size=10000)["hits"]["hits"][0]
        print(res)
        id=res["_id"]
        res=res["_source"]
        if ach_id=="Marathon":
            res["progress"]=0
        else:
            res["progress"]=df.steps.sum()
            print(df.steps.sum())
        if df.steps.sum()>=i*1000:
            res["achievedat"]=1
        print(res)
        es.update(index='achievements',id=id,doc=res)

In [ ]:
total_steps("user")

In [14]:
def avg_steps_per_week(userid):
    res = es.search(index="activity", query={'match' : {"userid":userid}},size=10000)["hits"]["hits"]
    res=list(map(lambda x: x["_source"],res))
    df=pd.DataFrame(res)
    df.start=df.start.apply(lambda x: datetime.fromtimestamp(x/1000))
    df.end=df.end.apply(lambda x: datetime.fromtimestamp(x/1000))
    # display(df[df.start > datetime.now() - pd.to_timedelta("7day")])
    for i in [5,6,8,10,12]:
        res = es.search(index="achievements", 
            query={"bool":{"must":[
                {'match' : 
                    {"userid":userid}},
                {'match' : 
                    {"achievementid":"AverageStepsWeek"+str(i)}}
            ]}},size=10000)["hits"]["hits"][0]
        id=res["_id"]
        res=res["_source"]
        average=df.steps.sum()/7
        res["progress"]=average
        if average>=i*1000:
            res["achievedat"]=1
        es.update(index='achievements',id=id,doc=res)

In [15]:
avg_steps_per_week("user")

In [30]:
def steps_in_a_day(userid):
    res = es.search(index="activity", query={'match' : {"userid":userid}},size=10000)["hits"]["hits"]
    res=list(map(lambda x: x["_source"],res))
    df=pd.DataFrame(res)
    df.start=df.start.apply(lambda x: datetime.fromtimestamp(x/1000))
    df.end=df.end.apply(lambda x: datetime.fromtimestamp(x/1000))
    max_steps_in_a_day=df.groupby(df.start.dt.date).steps.sum().max()
    for i in [10,12,14,16,18,20]:
        res = es.search(index="achievements", 
            query={"bool":{"must":[
                {'match' : 
                    {"userid":userid}},
                {'match' : 
                    {"achievementid":"DailyRecordSteps"+str(i)}}
            ]}},size=10000)["hits"]["hits"][0]
        id=res["_id"]
        res=res["_source"]
        res["progress"]=max_steps_in_a_day
        if max_steps_in_a_day>=i*1000:
            res["achievedat"]=1
            es.update(index='achievements',id=id,doc=res)
        else:
            break

In [29]:
steps_in_a_day("user")

{'_index': 'achievements', '_id': '2Ig21ZABlNYZt4JGnoYK', '_score': 3.4720056, '_source': {'userid': 'user', 'achievementid': 'DailyRecordSteps10', 'progress': 204000, 'goal': 10000, 'achievedat': 1}}
204000
{'_index': 'achievements', '_id': '2Yg21ZABlNYZt4JGnoYK', '_score': 3.4720056, '_source': {'userid': 'user', 'achievementid': 'DailyRecordSteps12', 'progress': 204000, 'goal': 12000, 'achievedat': 1}}
204000
{'_index': 'achievements', '_id': '2og21ZABlNYZt4JGnoYK', '_score': 3.4720056, '_source': {'userid': 'user', 'achievementid': 'DailyRecordSteps14', 'progress': 204000, 'goal': 14000, 'achievedat': 1}}
204000
{'_index': 'achievements', '_id': '24g21ZABlNYZt4JGnoYK', '_score': 3.4720056, '_source': {'userid': 'user', 'achievementid': 'DailyRecordSteps16', 'progress': 204000, 'goal': 16000, 'achievedat': 1}}
204000
{'_index': 'achievements', '_id': '3Ig21ZABlNYZt4JGnoYK', '_score': 3.4720056, '_source': {'userid': 'user', 'achievementid': 'DailyRecordSteps18', 'progress': 204000, 

In [47]:
def daily_goal_met(userid):
    res = es.search(index="activity", query={'match' : {"userid":userid}},size=10000)["hits"]["hits"]
    res=list(map(lambda x: x["_source"],res))
    df=pd.DataFrame(res)
    df.start=df.start.apply(lambda x: datetime.fromtimestamp(x/1000))
    df.end=df.end.apply(lambda x: datetime.fromtimestamp(x/1000))
    sums_days=df.groupby(df.start.dt.date).steps.sum()
    goals=list(map(lambda x:{"start":x["_source"]["start"],"end":x["_source"]["end"],"goal":x["_source"]["plan"]["activity"]["goal"]},es.search(index="plan", query={'match' : {"userid":"user"}},sort=[
    {
      "start": {
        "order": "desc"
      }
    }
  ],size=10000)["hits"]["hits"]))#[-1]
    display(df)
    print(goals)
    # num_reached=0
    # for goal in goals:
    #     temp_df=df.loc[(df.start>goal["start"]) & (df.end<goal["end"])]
    #     if temp_df.steps.sum()>=goal["goal"]:
    #         num_reached+=1

    # for i in [1,3,7,14,25,40,60,80,100]:
    #     res = es.search(index="achievements", 
    #             query={"bool":{"must":[
    #                 {'match' : 
    #                     {"userid":userid}},
    #                 {'match' : 
    #                     {"achievementid":"DailyGoalSteps"+str(i)}}
    #             ]}},size=10000)["hits"]["hits"][0]
    #     id=res["_id"]
    #     res=res["_source"]
    #     res["progress"]=num_reached
    #     if num_reached>=i:
    #         res["achievedat"]=1
    #     print(res)
        # es.update(index='achievements',id=id,doc=res)

In [48]:
daily_goal_met("user")

,start,end,type,steps,userid
0,2024-07-21 13:40:00,2024-07-21 14:28:00,current,14000,user
1,2024-07-15 10:00:00,2024-07-15 11:00:00,walking,200,user
2,2024-07-16 11:00:00,2024-07-16 12:00:00,walking,196,user
3,2024-07-18 10:00:00,2024-07-18 11:00:00,walking,2000,user
4,2024-07-18 03:00:00,2024-07-18 04:00:00,walking,400,user
5,2024-07-19 11:00:00,2024-07-19 12:00:00,walking,5679,user
6,2024-07-20 15:00:00,2024-07-20 16:00:00,walking,2000,user
7,2024-07-21 14:00:00,2024-07-21 15:00:00,walking,6000,user
8,2024-07-21 13:00:00,2024-07-21 14:00:00,walking,8000,user
9,2024-08-12 15:01:00,2024-08-12 15:02:00,current,102000,user


[{'start': '2024-08-12T11:55:24.206245', 'end': '2024-08-19T23:59:59.999999', 'goal': 3000}]


In [ ]:
#nå daglig mål for skritt(på rad) + for en hel uke
#3,7,14,25,40

In [59]:
goals=list(map(lambda x:{"start":x["_source"]["start"],"end":x["_source"]["end"],"goal":x["_source"]["plan"]["activity"]["goal"]},es.search(index="plan", query={'match' : {"userid":"user"}},size=10000)["hits"]["hits"]))#[-1]



reached=True
days_in_a_row=0
initial_date=datetime.now().date()
while reached:


{'start': '2024-07-21T11:43:50.657581',
 'end': '2024-07-28T23:59:59.999999',
 'goal': 3000}

In [ ]:
#gjennomsnittlig antall skritt på en uke
#5,6,8,10,12